In [1]:
!nvidia-smi

Fri Aug 18 10:38:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
! pip install -q transformers[sentencepiece] fastbook fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
! pip install -q onnxruntime onnx==1.14.0 onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *
from tqdm.notebook import tqdm
import numpy as np
import json

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/New folder

/content/drive/MyDrive/New folder


In [8]:
df = pd.read_csv("final_details.csv")


In [9]:
df=df.drop_duplicates(subset=['url'])
df = df[df.types != '[]']
df.describe()

,title,url,description,types
count,18397,18397,18396,18397
unique,17332,18397,14719,17254
top,Paella,https://eatsmarter.com/recipes/cucumber-sandwiches-with-fresh-goat-cheese,30 mins,"['None', 'low-carb']"
freq,24,1,320,157


In [10]:
df = df.dropna().reset_index(drop=True)
df.shape

(18396, 4)

In [11]:
import ast

for i in range(len(df.types)):
  try:
    list_1 = ast.literal_eval(df.types[i])
    if isinstance(list_1, list):
      df.types[i] = list_1
  except (ValueError, SyntaxError):
    pass

In [12]:
 types_list = df.types.to_list()

In [13]:
mapping = {
    'Onion': 'Vegetable',
    'Poultry': 'Chicken',
    'saltwater\xa0Fish': 'saltwater fish',
    'Root Vegetable': 'Vegetable'
}

for sublist in types_list:
    for i in range(len(sublist)):
        sublist[i] = mapping.get(sublist[i], sublist[i])

types_list[1]

['Finger Food',
 'Low-cholesterol',
 'Fitness',
 'Healthy Eating',
 'Low-calorie',
 'Low-calorie Meat Dish',
 '250-400 Calorie',
 'low-carb',
 'non-alcoholic',
 'egg-free',
 'Pescetarian',
 'Special',
 '25-Minute',
 'Buffet',
 'Exotic',
 'Cooking on vacation',
 'festively',
 'Meal for Four',
 'Garden Party',
 'Guest',
 'Party',
 'Picnic',
 'Cooking',
 'Fish',
 'Vegetable',
 'leafy green vegetable',
 'Vegetable',
 'Herb',
 'Dairy',
 'Fruit',
 'Brunch',
 'Snack',
 'Party Snack',
 'Party Snack',
 'Tapas',
 'Appetizer']

In [14]:
for sublist in types_list:
    new_sublist = []
    for item in sublist:
        if item not in ['Party Snack','Pescetarian','Picnic','Garden Party','Party','Stomach Discomfort','everyday meal','Elevated Blood Lipid','Crohn&#039;s Disease','European','Course','Main Course''Meal for Four','Menu', 'Guest', 'Easy', 'Lactation', 'Quick','Cooking','Osteoarthritis','Stress','Iron Deficiency','Cooking on vacation','Pregnancy','Fertility']:
            new_sublist.append(item)
    sublist[:] = new_sublist
types_list[1]

['Finger Food',
 'Low-cholesterol',
 'Fitness',
 'Healthy Eating',
 'Low-calorie',
 'Low-calorie Meat Dish',
 '250-400 Calorie',
 'low-carb',
 'non-alcoholic',
 'egg-free',
 'Special',
 '25-Minute',
 'Buffet',
 'Exotic',
 'festively',
 'Meal for Four',
 'Fish',
 'Vegetable',
 'leafy green vegetable',
 'Vegetable',
 'Herb',
 'Dairy',
 'Fruit',
 'Brunch',
 'Snack',
 'Tapas',
 'Appetizer']

In [15]:
for i in range (len(types_list)):
  types_list[i]=types_list[i][:5]

types_list[1]

['Finger Food', 'Low-cholesterol', 'Fitness', 'Healthy Eating', 'Low-calorie']

In [16]:
for sublist in types_list:
    sublist[:] = list(set(sublist))
types_list[1]

['Low-calorie', 'Healthy Eating', 'Finger Food', 'Low-cholesterol', 'Fitness']

In [17]:
df.types=types_list

In [18]:
df

,title,url,description,types
0,Cucumber Sandwiches with Fresh Goat Cheese,https://eatsmarter.com/recipes/cucumber-sandwiches-with-fresh-goat-cheese,"Due to the high water content, the vegetables are very helpful for weight loss. 100 grams of cucumber provide just 12 calories. Cucumbers also contain potassium, iron, zinc, fluorine, and some vitamin C. With the goat's cheese comes satiating protein between the slices.","[Vegetable, low-carb, Cheese]"
1,Smoked Trout Tartare with Avocado Mango Salsa,https://eatsmarter.com/recipes/smoked-trout-tartare-with-avocado-mango-salsa,"The sulfur substances contained in chives are responsible for their slight pungency. They provide a slight antibacterial effect and have a positive effect on digestion. If things run smoothly in the intestines, our concentration is also ensured. This is also supported by the omega-3 fatty acids from the smoked salmon trout.","[Low-calorie, Healthy Eating, Finger Food, Low-cholesterol, Fitness]"
2,Basic Tofu Recipe,https://eatsmarter.com/recipes/basic-tofu-recipe,"Do you have a sensitive stomach? Then tofu is perfect for you, because it's easy on the stomach and doesn't put a strain on digestion. Great veggie option: With around 15 percent vegetable protein, soy cheese gives us about as much as most types of meat offer.","[High-protein, nutritional, Healthy Eating, Low-cholesterol, Tofu]"
3,Grilled Watermelon with Herbed Ricotta,https://eatsmarter.com/recipes/grilled-watermelon-with-herbed-ricotta,"Despite the fruity-sweet taste of Watermelons there is relatively little sugar in it. On top, the fruit provides plenty of liquid for a refreshing feast on hot summer days. Essential oils from Cress help to get disease-causing bacteria under control. For example, cold symptoms can subside more quickly.","[Clean Eating, Gluten-free, nutritional, Healthy Eating, vegetarian grilling]"
4,Egg Bites with Smoked Salmon and Broccoli,https://eatsmarter.com/recipes/egg-bites-with-smoked-salmon-and-broccoli,"Eggs contain a lot of biotin. This B vitamin promotes hair growth and a healthy nail bed. In addition, the substance is needed for all metabolic processes in the organism.","[muffin, Gluten-free, nutritional, pastry, Baking]"
...,...,...,...,...
18391,Sausage and Ham Croissants,https://eatsmarter.com/recipes/sausage-and-ham-croissants,Roll the dough on a lightly floured surface to 4-5 mm (approximately 1/8 inch) thick. Cut into 3 cm (approximately 1 1/4 inch) wide and 20 cm (approximately 8 inch) long strips.,"[Pork, Quark-oil Dough, Milk, Breakfast, Eggs]"
18392,Shakshuka with Eggplant,https://eatsmarter.com/recipes/shakshuka-with-eggplant,This delicious vegetable and egg shakshuka dish is packed with a slew of vitamins and minerals thanks to the vegetable mixture as well as powerful protein from the eggs. Shakshukas are versatile and are a complete meal,"[baked dish, skillet dish, soul food, Eggs, skillet vegetables]"
18393,Currant Jelly,https://eatsmarter.com/recipes/currant-jelly-1,"Remove the jelly from the pan, and place on a long platter. Serve with bread and cheese.","[Low-fat, Gluten-free, Jelly, lactose-free, Low-cholesterol]"
18394,Multi-Grain Walnut Bread,https://eatsmarter.com/recipes/multi-grain-walnut-bread,"Add both flours, the shredded wheat, ground walnuts and salt and knead with the dough hook until smooth. Cover and let rise in a warm place until doubled in bulk, about 45 minutes.","[Yeast Dough, Bread, Bread Dough, Rustic, Baking]"


In [19]:
types_list = df.types.to_list()
type_count = {}
for type1 in types_list:
  if isinstance(type1, list):
    type_list = type1
  else:
    type_list = eval(type1)
  for type2 in type_list:
    if type2 in type_count.keys():
      type_count[type2] += 1
    else:
      type_count[type2] = 1
print(f"Number of types: {len(type_count)}")
print(type_count)

Number of types: 987
{'Vegetable': 5059, 'low-carb': 2131, 'Cheese': 541, 'Low-calorie': 79, 'Healthy Eating': 871, 'Finger Food': 158, 'Low-cholesterol': 719, 'Fitness': 88, 'High-protein': 1005, 'nutritional': 959, 'Tofu': 65, 'Clean Eating': 834, 'Gluten-free': 566, 'vegetarian grilling': 16, 'muffin': 77, 'pastry': 504, 'Baking': 1261, 'high-fiber': 1083, 'High-fiber Snack': 80, 'Snack': 257, 'Vegan': 344, 'Snack for Kids': 14, 'High-fiber Soup': 79, 'Stew': 247, 'diet': 1158, 'High-fiber Fish Dish': 99, 'Breakfast': 136, 'Low Carb Diet': 55, 'low-carb lunch': 43, 'low-carb dinner': 53, 'Alkaline': 19, 'cucumber salad': 28, 'Low-cholesterol Salad': 85, 'Fish': 1606, 'Spring Dish': 56, 'Spring': 335, 'Paleo Diet': 294, 'Cooking technique': 17, 'High-fiber Vegetarian Dish': 481, 'Spread': 53, 'Beef Stroganoff': 4, 'Beef Ragout': 26, 'Beef': 463, 'Grandma&#039;s': 59, 'High-protein Fish Dish': 148, 'salad': 2066, 'Gluten-free Vegetarian Recipe': 230, 'Eggs': 652, 'Gluten-free Snack': 

In [21]:
threshold = int(len(df) * 0.02)
rare_types = [key for key, value in type_count.items() if value < threshold]
len(rare_types), rare_types[:10]

(938,
 ['Low-calorie',
  'Finger Food',
  'Fitness',
  'Tofu',
  'vegetarian grilling',
  'muffin',
  'High-fiber Snack',
  'Snack',
  'Vegan',
  'Snack for Kids'])

In [22]:
types_list
revised_types_list = []
indices_to_drop = []

for idx, types in enumerate(types_list):
  if isinstance(types, list):
    type_list = types
  else:
    type_list = eval(types)

  revised_types = []

  for type1 in type_list:
    if type1 not in rare_types:
      revised_types.append(type1)

  if len(revised_types) == 0:
    indices_to_drop.append(idx)
  else:
    revised_types_list.append(revised_types)

df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(17821, 4)

In [23]:
df['revised_types'] = revised_types_list

In [24]:
df.tail()

,title,url,description,types,revised_types
17816,Vegetable Scrambled Eggs on Rolls,https://eatsmarter.com/recipes/vegetable-scrambled-eggs-on-rolls,"Heat the margarine in a large skillet. Saute the pepper, onion and thyme. Add the zucchini and tomatoes and cook briefly. Pour in the beaten eggs and let the eggs thicken, occasionally stirring with a spatula to push the edges to the center.","[Vegetable, roll, Eggs, Breakfast, Scrambled Egg]","[Vegetable, Eggs]"
17817,Sausage and Ham Croissants,https://eatsmarter.com/recipes/sausage-and-ham-croissants,Roll the dough on a lightly floured surface to 4-5 mm (approximately 1/8 inch) thick. Cut into 3 cm (approximately 1 1/4 inch) wide and 20 cm (approximately 8 inch) long strips.,"[Pork, Quark-oil Dough, Milk, Breakfast, Eggs]",[Eggs]
17818,Shakshuka with Eggplant,https://eatsmarter.com/recipes/shakshuka-with-eggplant,This delicious vegetable and egg shakshuka dish is packed with a slew of vitamins and minerals thanks to the vegetable mixture as well as powerful protein from the eggs. Shakshukas are versatile and are a complete meal,"[baked dish, skillet dish, soul food, Eggs, skillet vegetables]","[baked dish, Eggs]"
17819,Currant Jelly,https://eatsmarter.com/recipes/currant-jelly-1,"Remove the jelly from the pan, and place on a long platter. Serve with bread and cheese.","[Low-fat, Gluten-free, Jelly, lactose-free, Low-cholesterol]","[Gluten-free, Low-cholesterol]"
17820,Multi-Grain Walnut Bread,https://eatsmarter.com/recipes/multi-grain-walnut-bread,"Add both flours, the shredded wheat, ground walnuts and salt and knead with the dough hook until smooth. Cover and let rise in a warm place until doubled in bulk, about 45 minutes.","[Yeast Dough, Bread, Bread Dough, Rustic, Baking]",[Baking]


In [25]:
revised_types_list = df.revised_types.to_list()
revised_types_list

[['Vegetable', 'low-carb', 'Cheese'],
 ['Healthy Eating', 'Low-cholesterol'],
 ['High-protein', 'nutritional', 'Healthy Eating', 'Low-cholesterol'],
 ['Clean Eating', 'Gluten-free', 'nutritional', 'Healthy Eating'],
 ['Gluten-free', 'nutritional', 'pastry', 'Baking'],
 ['high-fiber', 'Clean Eating', 'nutritional'],
 ['Clean Eating', 'nutritional', 'low-carb'],
 ['high-fiber', 'diet'],
 ['High-protein', 'Gluten-free', 'diet'],
 ['nutritional', 'Healthy Eating', 'low-carb'],
 ['Clean Eating', 'Low-cholesterol'],
 ['Fish', 'low-carb', 'Low-cholesterol'],
 ['high-fiber', 'Fish', 'diet'],
 ['high-fiber', 'Clean Eating', 'nutritional', 'High-fiber Vegetarian Dish'],
 ['Clean Eating', 'Gluten-free', 'nutritional', 'Low-cholesterol'],
 ['Beef', 'low-carb'],
 ['High-protein', 'Clean Eating', 'nutritional', 'salad'],
 ['Gluten-free', 'nutritional', 'Eggs'],
 ['Fish', 'Low-cholesterol'],
 ['high-fiber', 'Fish', 'Low-cholesterol'],
 ['Clean Eating', 'Healthy Eating'],
 ['High-protein', 'Clean Eati

In [26]:
revised_types_list
revised_type_count = {}
for types in revised_types_list:
  type_list = types
  for type1 in type_list:
    if type1 in revised_type_count.keys():
      revised_type_count[type1] += 1
    else:
      revised_type_count[type1] = 1
print(f"Number of types: {len(revised_type_count)}")
print(revised_type_count)

Number of types: 49
{'Vegetable': 5059, 'low-carb': 2131, 'Cheese': 541, 'Healthy Eating': 871, 'Low-cholesterol': 719, 'High-protein': 1005, 'nutritional': 959, 'Clean Eating': 834, 'Gluten-free': 566, 'pastry': 504, 'Baking': 1261, 'high-fiber': 1083, 'diet': 1158, 'Fish': 1606, 'High-fiber Vegetarian Dish': 481, 'Beef': 463, 'salad': 2066, 'Eggs': 652, 'Vegetarian': 743, 'saltwater fish': 1054, 'fruit-vegetable': 2156, 'flowering vegetables': 469, 'Vegetable Salad': 845, 'Meat': 1858, 'Sauce': 489, 'Asian': 821, 'Chicken': 1349, 'Dessert': 1477, 'Herb': 1709, 'baked dish': 772, 'Legume': 507, 'Roast': 821, 'Shrimp': 564, 'Italian': 719, 'soup': 537, 'Seafood': 549, 'non-alcoholic': 433, 'Casserole': 612, 'mushroom': 408, 'Spices': 1054, 'leafy green vegetable': 559, 'Sprout': 544, 'Fruit': 1101, 'Tropical Fruit': 453, 'Dough': 1901, 'Pome Fruit': 462, 'Pasta': 2271, 'Rice': 1344, 'Fruit Dessert': 368}


In [27]:
revised_type_count.keys()

dict_keys(['Vegetable', 'low-carb', 'Cheese', 'Healthy Eating', 'Low-cholesterol', 'High-protein', 'nutritional', 'Clean Eating', 'Gluten-free', 'pastry', 'Baking', 'high-fiber', 'diet', 'Fish', 'High-fiber Vegetarian Dish', 'Beef', 'salad', 'Eggs', 'Vegetarian', 'saltwater fish', 'fruit-vegetable', 'flowering vegetables', 'Vegetable Salad', 'Meat', 'Sauce', 'Asian', 'Chicken', 'Dessert', 'Herb', 'baked dish', 'Legume', 'Roast', 'Shrimp', 'Italian', 'soup', 'Seafood', 'non-alcoholic', 'Casserole', 'mushroom', 'Spices', 'leafy green vegetable', 'Sprout', 'Fruit', 'Tropical Fruit', 'Dough', 'Pome Fruit', 'Pasta', 'Rice', 'Fruit Dessert'])

In [28]:
encode_types = { key: idx for idx, (key, value) in enumerate(revised_type_count.items())}
with open("recipe5_types_encoded.json", "w") as fp:
  json.dump(encode_types, fp)

In [29]:
categorical_type_list = []
revised_types_list

for type5 in revised_types_list:
  categorical_list = [0] * len(encode_types)
  for type1 in type5:
    type1_index = encode_types[type1]
    categorical_list[type1_index] = 1
  categorical_type_list.append(categorical_list)

categorical_type_list[3][:45]

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [30]:
df['type_cat_list'] = categorical_type_list
df.head()

,title,url,description,types,revised_types,type_cat_list
0,Cucumber Sandwiches with Fresh Goat Cheese,https://eatsmarter.com/recipes/cucumber-sandwiches-with-fresh-goat-cheese,"Due to the high water content, the vegetables are very helpful for weight loss. 100 grams of cucumber provide just 12 calories. Cucumbers also contain potassium, iron, zinc, fluorine, and some vitamin C. With the goat's cheese comes satiating protein between the slices.","[Vegetable, low-carb, Cheese]","[Vegetable, low-carb, Cheese]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Smoked Trout Tartare with Avocado Mango Salsa,https://eatsmarter.com/recipes/smoked-trout-tartare-with-avocado-mango-salsa,"The sulfur substances contained in chives are responsible for their slight pungency. They provide a slight antibacterial effect and have a positive effect on digestion. If things run smoothly in the intestines, our concentration is also ensured. This is also supported by the omega-3 fatty acids from the smoked salmon trout.","[Low-calorie, Healthy Eating, Finger Food, Low-cholesterol, Fitness]","[Healthy Eating, Low-cholesterol]","[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Basic Tofu Recipe,https://eatsmarter.com/recipes/basic-tofu-recipe,"Do you have a sensitive stomach? Then tofu is perfect for you, because it's easy on the stomach and doesn't put a strain on digestion. Great veggie option: With around 15 percent vegetable protein, soy cheese gives us about as much as most types of meat offer.","[High-protein, nutritional, Healthy Eating, Low-cholesterol, Tofu]","[High-protein, nutritional, Healthy Eating, Low-cholesterol]","[0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Grilled Watermelon with Herbed Ricotta,https://eatsmarter.com/recipes/grilled-watermelon-with-herbed-ricotta,"Despite the fruity-sweet taste of Watermelons there is relatively little sugar in it. On top, the fruit provides plenty of liquid for a refreshing feast on hot summer days. Essential oils from Cress help to get disease-causing bacteria under control. For example, cold symptoms can subside more quickly.","[Clean Eating, Gluten-free, nutritional, Healthy Eating, vegetarian grilling]","[Clean Eating, Gluten-free, nutritional, Healthy Eating]","[0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Egg Bites with Smoked Salmon and Broccoli,https://eatsmarter.com/recipes/egg-bites-with-smoked-salmon-and-broccoli,"Eggs contain a lot of biotin. This B vitamin promotes hair growth and a healthy nail bed. In addition, the substance is needed for all metabolic processes in the organism.","[muffin, Gluten-free, nutritional, pastry, Baking]","[Gluten-free, nutritional, pastry, Baking]","[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [31]:
labels = list(encode_types.keys())

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

In [33]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,url,description,types,revised_types,type_cat_list
7156,Chickpea Tofu Meatballs on Bulgur Salad,https://eatsmarter.com/recipes/chickpea-tofu-meatballs-on-bulgur-salad,It's the mix that does it: Chickpeas and tofu are very high in fiber and contain plenty of blood-building iron and cell-protecting vitamin E.,"[Vegan, Vegan Lunch, Clean Eating, nutritional, Healthy Eating]","[Clean Eating, nutritional, Healthy Eating]","[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6828,Tomato Bread Salad with Feta and Olives,https://eatsmarter.com/recipes/tomato-bread-salad-with-feta-and-olives,"Feta, a type of sheep's milk cheese, is rich in protein and helps support strong bones with an ideal mix of calcium and phosphorus. In addition, it helps boost the immune system with the mineral zinc.","[Summer Dish, Summer Vegetable, Vegetarian, Summer, Summer Salad]",[Vegetarian],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9571,Vegan Tagliatelle with Squash and Shallots,https://eatsmarter.com/recipes/vegan-tagliatelle-with-squash-and-shallots,45 mins,"[squash, flowering vegetables, Seed, Vegetable, Pasta]","[flowering vegetables, Vegetable, Pasta]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
5729,"Salad with Eggs, Bacon, and Peanuts",https://eatsmarter.com/recipes/salad-with-eggs-bacon-and-peanuts,"Rinse and dry the parsley, then remove the leaves from the stems. Toast the peanuts in a dry pan.","[Eggs, Egg Salad, Vegetable Salad, low-carb]","[Eggs, Vegetable Salad, low-carb]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9112,Simple Cheese Pasta,https://eatsmarter.com/recipes/simple-cheese-pasta,10 mins,"[Dough, Pasta, Cheese]","[Dough, Pasta, Cheese]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]"


In [34]:
model_path = "/content/drive/MyDrive/New folder/recipe-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [35]:
learner_inf.blurr_predict("random placeholder")

[{'labels': [],
  'scores': [],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Vegetable', 'low-carb', 'Cheese', 'Healthy Eating', 'Low-cholesterol', 'High-protein', 'nutritional', 'Gluten-free', 'Clean Eating', 'Baking', 'pastry', 'high-fiber', 'diet', 'Fish', 'High-fiber Vegetarian Dish', 'Beef', 'salad', 'Eggs', 'Vegetarian', 'fruit-vegetable', 'saltwater fish', 'Vegetable Salad', 'flowering vegetables', 'Meat', 'Sauce', 'Asian', 'Chicken', 'Dessert', 'Herb', 'baked dish', 'Roast', 'Legume', 'Shrimp', 'Italian', 'soup', 'Seafood', 'non-alcoholic', 'Casserole', 'mushroom', 'Spices', 'leafy green vegetable', 'Sprout', 'Fruit', 'Tropical Fruit', 'Dough', 'Pome Fruit', 'Pasta', 'Rice', 'Fruit Dessert'],
  '

In [36]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

[]

In [40]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['type_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [38]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_types = [0] * len(encode_types)
  for label in labels:
    pred_types[encode_types[label]] = 1
  preds.append(pred_types)

preds[0][:20]

  0%|          | 0/1782 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [41]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.24747231584015406
F1 Score (Macro) = 0.18307236605175722


In [42]:
model_path = "/content/drive/MyDrive/New folder/recipe-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [43]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    '/content/drive/MyDrive/New folder/recipe-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [44]:
!pip install -q onnxruntime.quantization

ERROR: Could not find a version that satisfies the requirement onnxruntime.quantization (from versions: none)
ERROR: No matching distribution found for onnxruntime.quantization


In [45]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = '/content/drive/MyDrive/New folder/recipe-classifier.onnx'
quantized_onnx_model_path = '/content/drive/MyDrive/New folder/recipe-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

In [46]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_types.keys())

inf_session = rt.InferenceSession('/content/drive/MyDrive/New folder/recipe-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [47]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_types)
  for label in labels:
    pred_genres[encode_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/1782 [00:00<?, ?it/s]

In [49]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.2312008978675645
F1 Score (Macro) = 0.16719754141014162


In [50]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_types.keys())

inf_session = rt.InferenceSession('/content/drive/MyDrive/New folder/recipe-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [51]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_types)
  for label in labels:
    pred_genres[encode_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/1782 [00:00<?, ?it/s]

In [52]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.22909725527526575
F1 Score (Macro) = 0.15900059000627506


NameError: ignored